In [25]:
import cv2
import numpy as np

In [26]:
img = cv2.imread("/home/mericdemirors/Pictures/araba/araba.jpg")

In [ ]:
segmented_image = img

In [34]:
cv2.imshow("i", img)
cv2.imshow("im", segmented_image.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [29]:
cv2.destroyAllWindows()